[I BYTES](#BYTES)

- [I.1 Les bytes en python](#Bytes_en_python)

- [I.2 Module struct : conversion de nombres python en bytes langage C](#Module_struct)

- [I.3 Représenter les bytes en base b](#bytes_en_base_b)

- [I.4 Module ctypes : pour aller plus loin que struct](#Module_ctypes)

[II ENCODAGES : GENERALITES](#ENCODAGES_GENERALITES)

- [II.1 Définitions](#encodages_definitions)

- [II.2 Exemples de caractères étrangers et liste des encodages python](#caracteres_etrangers)

- [II.3 ascii, ansi, iso 8859-1, latin_1, utf-8 ... par Wikipédia](#encodages_usuels)

[III ENCODAGES EN PYTHON](#ENCODAGES_PYTHON)

- [III.1 jouer avec unicode : `ord()   chr()   \N   \u   \U`](#unicode)

- [III.2 jouer avec utf-8 : `encode()` et `decode()`](#utf-8)

- [III.3 jouer avec `unicodedata`](#unicodedata)

- [III.4 mises en garde lors de la manipulation de caractères étrangers ...](#mise_en_garde_caracteres_etrangers)

 

 

entiers définis en octal, hexadécimal, binaire ... : dans I.1

IEE754 : dans I.2


<a id="BYTES"></a>
# I BYTES

<a id='Bytes_en_python'></a>
## I.1 Les bytes en python

Essentiellement, les bytes sont des séquences immutables d'octets (pour des séquences mutables : [bytearrays](https://docs.python.org/3.6/library/stdtypes.html#bytearray) ) où les octets (c'est à dire les éléments de la séquence) sont de type `int`.

**Remarque** : lorsqu'on demande à faire un print sur un objet `bytes`, si un des octets correspond au code ascii d'un caractère imprimable (de 32 à 126, plus 9, 10 et 13), alors il sera affiché comme le caractère ascii correspondant.

### I.1.a) Définir un <code>bytes</code> à partir d'une Literal

On utilise une [literals](https://docs.python.org/3/reference/lexical_analysis.html#literals) avec le préfixe `b` (il y a les String literals et les Bytes literals, le préfixe `b` correspond bien entendu à une Bytes literal).

La façon la plus naturelle pour le faire est d'utiliser des octets codés en héxadécimal. Dans ce cas l'expression littérale utilisera fortement le préfixe `\x` qui indique que les deux caractères suivants correspondent au codage d'un octet en hexadécimal.

In [ ]:
mes_bytes = b'\xff\xa7'
print(mes_bytes)
print(mes_bytes[0])
print(type(mes_bytes))
print(type(mes_bytes[0]))

**Il faut néanmoins impérativement avoir en tête que les octets correspondant à des codes ascii de caractères imprimables sont automatiquement considérés comme des caractères dans les literals**, tant lors de la lecture que lors de l'affichage écran. Dès lors voici ce qu'on peut faire (le code ascii `\x61 = 97` correspond à `'a'`, le code ascii `\x62 = 98` correspond à `'b'` ...):

In [ ]:
mes_bytes = b'\xffab\x61'
print(mes_bytes)
for b in mes_bytes:
    print(b)

On remarque sur cet exemple que, lorsqu'on affiche un élément unique de la séquence `bytes`, la conversion en caractère n'est pas effectuée : c'est l'entier en base 10 correspondant à l'octet qui est affiché.

**Attention :** On prendra garde que, lors de la définition d'un  `bytes` avec des literals, les espaces sont considérés comme des octets :

In [1]:
mes_bytes = b'\xff \x61'
print(mes_bytes)

b'\xff a'


**Attention :** `0x` permet de définir des entiers en hexadécimal alors que `\x` permet de définir un caractère défini en héxadécimal.

In [ ]:
x = 0x64
y = '\x64'
z = 0o144
w = 0b1100100
print(x)
print(y)
print(z)
print(w)

**Remarque :** On note au passage que `print()` sur un entier l'affiche par défaut en décimal, quand bien il a été défini en héxadécimal, en octal ou en binaire. Pour l'afficher dans une autre base, il faudrait utiliser `format()`.

### I.1.b) Définir un `bytes` à partir d'une chaîne de caractères représentant une notation hexadécimale

Cette fois-ci on évite le cas où python affiche les octets correspondant à des codes ascii de caractères imprimables (de 32 à 126, plus 9, 10 et 13) sous forme de caractères.

In [ ]:
ma_chaine_hexa = '63f061 F1ff  '
mes_bytes = bytes.fromhex(ma_chaine_hexa)
print(mes_bytes)
for b in mes_bytes:
    print(b)

In [ ]:
mes_bytes = b'abcd'
ma_chaine_hexa = mes_bytes.hex()
print(ma_chaine_hexa)
for b in mes_bytes:
    print(b)

### I.1.c) Définir un `bytes` en utilisant une conversion de type

Cela peut se faire à partir :

- d'une chaîne de caractères. Dans ce cas il faut préciser l'encodage désiré (la chaîne sera convertie en utilisant la méthode `str.encode()`).

- d'un itérable d'entiers. Dans ce cas les entiers fournis doivent être compris entre 0 et 255.

- d'un objet conforme à l'interface `buffer` (exemple non traité ici).

On peut aussi initialiser un `bytes` d'octets nuls en passant en argument le nombre d'octets nuls que l'on souhaites initialiser.

In [ ]:
mes_bytes = bytes('abcdef ç é ù', 'utf-8')
print(mes_bytes)

In [ ]:
mes_bytes = bytes([0, 255, 64, 129])
print(mes_bytes)

In [ ]:
mes_bytes = bytes(10)
print(mes_bytes)

<div class="alert alert-info">
### En résumé 
- Une `bytes literal` peut être utilisée pour définir un objet `bytes`. Dans cette `bytes literal`, on utilise soit des caractères ascii imprimables soit des entiers en notation hexadécimale grâce au préfixe `\x`. 

- Lors de l'affichage grâce à print, les octets correspondant à des codes ascii de caractères imprimables (de 32 à 126, plus 9, 10 et 13) seront automatiquement affichés sous forme de caractères.

- En revanche lors de l'affichage d'un seul élément d'un `bytes`, l'octet est affiché sous la forme d'un entier entre 0 et 255.

- En utilisant `hex()` et `fromhex` on peut aussi faire des conversion en bytes à partir de chaînes de caractères désignant les octets écrits en hexadécimal mais sans aucun préfixe.

- On peut aussi utiliser des conversions de type à partir de chaînes de caractères (ce qui utilise `encode()` en arrière-plan) ou à partir d'itérables d'entiers compris entre 0 et 255.

<a id='Module_struct'></a>
# I.2 struct : conversion nombres python en bytes langage C

- This module performs conversions between Python values and C structs represented as Python bytes objects. This can be used in handling binary data stored in files or from network connections, among other sources. It uses Format Strings as compact descriptions of the layout of the C structs and the intended conversion to/from Python values.

- **Attention :** il semble que ces conversions soient *abstraites* au sens où les entiers python, par exemple, prennent plus d'octets en mémoire que les conversions effectuées via ce module. Voir `ctypes` ci-dessous.

Le premier caractère de la chaîne de formatage correspond à :

![page module struct python](ressources/struct_1.jpg)

Le caractère suivant correspond au type à packer (on note que le type `char` en langage C correspond à un type de nombre).

![page module struct python](ressources/struct_2.jpg)



In [2]:
from struct import pack
mes_bytes = pack('!b', 127)
print(mes_bytes)

b'\x7f'


In [3]:
mes_bytes = pack('!I', 1234584789)
print(mes_bytes)

b'I\x96D\xd5'


### Le cas des flottants : accéder à la norme IEE754

Une fois que l'on maîtrise ce qui a été fait ci-dessus, ce n'est pas bien compliqué. Néanmoins il faut prendre garde au big-endian vs little-endian (il semble qu'il faille du big-endian : changer le signe de `x` pour s'en convaincre).

In [24]:
x = 64.875
mes_bytes = pack('>f', x)
s_b = ''
s_x = ''
for b in mes_bytes:
    s_b = s_b + ' ' + '{:08b}'.format(b)
    s_x = s_x + ' ' + '{:02x}'.format(b)

print(s_b)
print(s_x)

 01000010 10000001 11000000 00000000
 42 81 c0 00


<div class="alert alert-info">
**Attention :** il semble que la méthode pack ne fonctionne pas pour des entiers arbitrairement longs (que python sait pourtant gérer). Pour fouiller cette notion, aller voir la section relative au module `ctypes`.

<a id='bytes_en_base_b'></a>
# I.3 Représenter les bytes en base b



In [ ]:
mes_bytes = b'\xa3\xb5\x91'
for b in mes_bytes:
    print('{:08b}'.format(b))
    print('{:02x}'.format(b))
    print('{:03d}'.format(b))
    print('---------')

In [ ]:
s = '10100011'
t = 'b5'
u = '145'
mes_bytes = bytes([int(s, 2), int(t, 16), int(u, 10)])
print(mes_bytes)

<a id = "Module_ctypes"></a>
# I.4 ctypes : pour aller plus loin que struct

La librairie `ctypes` permet d'accéder aux structures de données écrites en C qui sont utilisées par l'interpréteur python c'est-à-dire le logiciel qui exécute le code python. On dit qu'il l'interprète.  Par conséquent il est possible grâce à cette librairie d'explorer comme sont repésentés les entiers en Python.

Il est ainsi possible d'afficher par exemple une partie des données contenues dans la structure de données `PyLongObject`, laquelle permet de représenter les entiers en Python. 

Notamment, pour les grands entiers, on peut accéder aux octets alors qu'avec `struct` nous étions limités.

Il apparait que les entiers sont stockés sous forme de paquets de 30 bits ...

**Remarque :**

- `id(object)`
    - Return the “identity” of an object. This is an integer which is guaranteed to be unique and constant for this object during its lifetime. Two objects with non-overlapping lifetimes may have the same id() value.

    - CPython implementation detail: This is the address of the object in memory.
    

**Attention :**

Dans le code ci-dessous, le tableau px doit être parcouru de 1 à size compris (et non pas de 0 à size - 1).




In [ ]:
import ctypes

class PyLongObject(ctypes.Structure):
        _fields_ = [("ob_refcnt", ctypes.c_long),
                ("ob_type", ctypes.c_void_p),
                ("ob_size", ctypes.c_ulong),
                ("ob_digit", ctypes.c_int *1)]

bignum = 6*2**180+5*2**150+4*2**120+3*2**90+2*2**60+2**30+41
print(bignum)
the_int=PyLongObject.from_address(id(bignum))
size=the_int.ob_size


x=the_int.ob_digit  # récupère ob_digit qui est l'adresse d'un tableau d'entiers C  (c_int *)
px= ctypes.cast(ctypes.pointer(x),ctypes.POINTER(ctypes.c_int))# transforme  le tableau d'entiers C en tableau Python

recompose = 0
for i in range(1, size+1):
    print(px[i])
    recompose = recompose + px[i]*2**(30*(i-1))

print('------------------------')
print(recompose)
print(bignum)
#for p in px:
#    print(p, end = ' | ')

<a id="ENCODAGES_GENERALITES"></a>
# II ENCODAGES : GENERALITES

<a id = "encodages_definitions"></a>
## II.1 Definitions (from python doc)

### ----------------- EXTRAIT DE  WIKIPEDIA

The Unicode specifications are continually revised and updated to add new languages and symbols.

A character is the smallest possible component of a text. ‘A’, ‘B’, ‘C’, etc., are all different characters. So are ‘È’ and ‘Í’. Characters vary depending on the language or context you’re talking about. For example, there’s a character for “Roman Numeral One”, ‘Ⅰ’, that’s separate from the uppercase letter ‘I’. They’ll usually look the same, but these are two different characters that have different meanings.

The Unicode standard describes how characters are represented by code points. A code point value is an integer in the range 0 to 0x10FFFF (about 1.1 million values, with some 110 thousand assigned so far). In the standard and in this document, a code point is written using the notation U+265E to mean the character with value 0x265e (9,822 in decimal).

**Strictly, these definitions imply that it’s meaningless to say ‘this is character U+265E’. U+265E is a code point, which represents some particular character; in this case, it represents the character ‘BLACK CHESS KNIGHT’, ‘♞’. In informal contexts, this distinction between code points and characters will sometimes be forgotten.**

A character is represented on a screen or on paper by a set of graphical elements that’s called a glyph. The glyph for an uppercase A, for example, is two diagonal strokes and a horizontal stroke, though the exact details will depend on the font being used. Most Python code doesn’t need to worry about glyphs; figuring out the correct glyph to display is generally the job of a GUI toolkit or a terminal’s font renderer.

**A Unicode string is a sequence of code points, this sequence of code points needs to be represented in memory as a set of code units, and code units are then mapped to 8-bit bytes. The rules for translating a Unicode string into a sequence of bytes are called a character encoding, or just an encoding.**

The first encoding you might think of is using 32-bit integers as the code unit, and then using the CPU’s representation of 32-bit integers. This representation is straightforward but using it presents a number of problems.
- It’s not portable; different processors order the bytes differently.
- It’s very wasteful of space. In most texts, the majority of the code points are less than 127, or less than 255, so a lot of space is occupied by 0x00 bytes. Increased RAM usage doesn’t matter too much (desktop computers have gigabytes of RAM, and strings aren’t usually that large), but expanding our usage of disk and network bandwidth by a factor of 4 is intolerable.
- It’s not compatible with existing C functions such as strlen(), so a new family of wide string functions would need to be used.

Therefore this encoding isn’t used very much, and people instead choose other encodings that are more efficient and convenient, such as UTF-8.

**UTF-8 is one of the most commonly used encodings, and Python often defaults to using it. UTF stands for “Unicode Transformation Format”, and the ‘8’ means that 8-bit values are used in the encoding. (There are also UTF-16 and UTF-32 encodings, but they are less frequently used than UTF-8.) UTF-8 uses the following rules:**
- **If the code point is < 128, it’s represented by the corresponding byte value.**
- **If the code point is >= 128, it’s turned into a sequence of two, three, or four bytes, where each byte of the sequence is between 128 and 255.**

UTF-8 has several convenient properties:
- It can handle any Unicode code point.
- A Unicode string is turned into a sequence of bytes that contains embedded zero bytes only where they represent the null character (U+0000). This means that UTF-8 strings can be processed by C functions such as strcpy() and sent through protocols that can’t handle zero bytes for anything other than end-of-string markers.
- A string of ASCII text is also valid UTF-8 text.
- UTF-8 is fairly compact; the majority of commonly used characters can be represented with one or two bytes.
- If bytes are corrupted or lost, it’s possible to determine the start of the next UTF-8-encoded code point and resynchronize. It’s also unlikely that random 8-bit data will look like valid UTF-8.
- UTF-8 is a byte oriented encoding. The encoding specifies that each character is represented by a specific sequence of one or more bytes. This avoids the byte-ordering issues that can occur with integer and word oriented encodings, like UTF-16 and UTF-32, where the sequence of bytes varies depending on the hardware on which the string was encoded.



<div class="alert alert-info">

Pour résumer, Unicode est un jeu de caractères : il établit une correspondance entre des caractères et des `code points`.

Quant à utf-8, c'est l'encodage de ces `code points` en mémoire.

<a id="caracteres_etrangers"></a>
## II.2 Exemples de caractères étrangers et liste des encodages python

[Texte I can eat glass : une même phrase dans plein d'encodages](ressources/I_can_eat_glass.txt)

[Liste des encodages disponibles dans python 3.6](ressources/Liste_encodages.txt)

<a id="encodages_usuels"></a>
## II.3 ascii, ansi, iso 8859-1, latin_1, utf-8 ... par Wikipédia

https://fr.wikipedia.org/wiki/ISO/CEI_646#Variantes_nationales_de_l%E2%80%99ISO_646

Iso 646 (sur 7 bits)
- variante de base est l'US (ASCII)
- contient d'autres variantes nationales

https://stackoverflow.com/questions/701882/what-is-ansi-format

ANSI encoding is a slightly generic term used to refer to the standard code page on a system, usually Windows. It is more properly referred to as Windows-1252 on Western/U.S. systems. (It can represent certain other Windows code pages on other systems.) This is essentially an extension of the ASCII character set in that it includes all the ASCII characters with an additional 128 character codes. This difference is due to the fact that "ANSI" encoding is 8-bit rather than 7-bit as ASCII is (ASCII is almost always encoded nowadays as 8-bit bytes with the MSB set to 0). 

The name "ANSI" is a misnomer, since it doesn't correspond to any actual ANSI standard, but the name has stuck. ANSI is not the same as UTF-8.

https://fr.wikipedia.org/wiki/Windows-1252

Windows-1252 est une extension de l'ISO/CEI 8859-1 : il diffère du codage ISO-8859-1 par l'utilisation de caractères imprimables, plutôt que des caractères de contrôle, dans les codes 128 à 159. Pour les utilisateurs de Windows, Microsoft appelle ceci de manière générique ANSI, mais, en fonction de l'endroit où le système d'exploitation a été vendu, l'ensemble de caractères peut avoir un autre nom, comme CP1252 aux États-Unis ou, dans les pays de l'Europe de l'Ouest, le nom validé par l'IANA, Windows-1252. 

https://fr.wikipedia.org/wiki/ISO/CEI_8859-1

La norme ISO 8859-1, dont le nom complet est ISO/CEI 8859-1, et qui est souvent appelée Latin-1 ou Europe occidentale, forme la première partie de la norme internationale ISO/CEI 8859, qui est une norme de l’Organisation internationale de normalisation pour le codage des caractères en informatique.

Elle définit ce qu’elle appelle l’alphabet latin numéro 1, qui consiste en 191 caractères de l’alphabet latin, chacun d’entre eux étant codé sur un octet (soit 8 bits). ISO 8859-1 reprend le codage des caractères imprimables d’US-ASCII.

Dans les pays occidentaux, cette norme était utilisée par de nombreux systèmes d’exploitation, dont UNIX, Windows ou AmigaOS. Elle a donné lieu à quelques extensions et adaptations, dont Windows-1252 et ISO/CEI 8859-15.

https://fr.wikipedia.org/wiki/UTF-8#Interop%C3%A9rabilit%C3%A9

utf-8 permet de représenter les milliers de caractères du répertoire universel, commun à la norme ISO/CEI 10646 et au standard Unicode (du moins depuis sa version 1.1.1).
Un texte en US-ASCII est codé identiquement en UTF-8 (lorsque le BOM n’est pas utilisé). 


<a id="ENCODAGES_PYTHON"></a>
# III : ENCODAGES EN PYTHON

Rappelons ici que :
- struct.pack fonctionne pour obtenir des *nombres* en bytes, mais pas des caractères ... d'où `encode` et `decode`.
- les code point unicode sont compris entre 0x000000 et 0x10FFFF c'est à dire entre 0 et 1 114 111.

<a id = "unicode"></a>
## III.1 jouer avec unicode : `ord()   chr()   \N   \u   \U`

La méthode `ord()` permet d'obtenir le code point **décimal** d'un caractère donné :

In [ ]:
x = ord('a')
y = ord('ç')
z = ord('㒕')
print(x, y, z)

La méthode `chr()` permet d'obtenir le caractère correspondant à un code point en **écriture décimale** :

In [ ]:
x = chr(97)
y = chr(231)
z = chr(13461)
print(x, y, z)

On peut aussi insérer, dans une chaîne de caractères, n'importe quel caractère unicode grâce aux préfixes `\u` ou `\U` et au code point dudit caractère en **écriture héxadécimale** (pour les caractères ascii on peut se contenter de `\x` à condition d'être dans la plage 32 à 126, plus 9, 10 et 13) :

In [4]:
x = "\N{LATIN SMALL LETTER A}"     # Using the character name
y = '\u3495'                       # Using a 16-bit hex value
z = "\U00023F9A"                   # Using a 32-bit hex value
zz = "\x41"
print(x, y, z, zz)

a 㒕 𣾚 A


<a id = "utf-8"></a>
## III.2 jouer avec utf-8 : `encode()` et `decode()`

In [7]:
ma_chaine = 'abcdef à ù ç ට 㒕 𣾚'
mes_bytes = ma_chaine.encode('utf-8')
print(mes_bytes)

b'abcdef \xc3\xa0 \xc3\xb9 \xc3\xa7 \xe0\xb6\xa7 \xe3\x92\x95 \xf0\xa3\xbe\x9a'


In [10]:
mes_bytes = b'abcdef \xc3\xa0 \xc3\xb9 \xc3\xa7 \xe0\xb6\xa7 \xe3\x92\x95 \xf0\xa3\xbe\x9a'
ma_chaine = mes_bytes.decode('utf-8')
print(ma_chaine)

abcdef à ù ç ට 㒕 𣾚


<a id = "unicodedata"></a>
## III.3 jouer avec `unicodedata`

In [11]:
import unicodedata

c = chr(97)

print(unicodedata.category(c), unicodedata.name(c))
print(c)

Ll LATIN SMALL LETTER A
a


<a id="mise_en_garde_caracteres_etrangers"></a>
## III.4 mises en garde lors de la manipulation de caractères étrangers ...

Regardons les 3 exemples ci-dessous :

In [12]:
ma_chaine = 'ती'
mes_bytes = ma_chaine.encode('utf-8')
print(mes_bytes)

b'\xe0\xa4\xa4\xe0\xa5\x80'


In [13]:
mes_bytes = b'\xe0\xa4\xa4'
ma_chaine = mes_bytes.decode('utf-8')
print(ma_chaine)

त


In [14]:
mes_bytes = b'\xe0\xa5\x80'
ma_chaine = mes_bytes.decode('utf-8')
print(ma_chaine)

ी


Ci-dessus un exemple de caractère qui nécessite un autre caractère devant.
Lorsqu'on cherche à copier-coller le caractère, en fait on copie-colle deux caractères à la fois...

**Culture générale :** (tiré de https://blogs.transparent.com/hindi/dependent-vowels/)

In Hindi, the vowels have a dependent form called matra (मात्रा ). Matras (मात्रा ) are vowels that cannot be written alone. They have to be accompanied with a consonant. Matras (मात्रा ) are different from independent vowels because independent vowels can be written alone.

On pensera aussi aux propriétés suivantes de certains encodages : unicode.bidirectionnal, unicodedata.mirrored, unicodedata.combining, unicodedata.decomposition ...